In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score



In [2]:
sheet='Engineering & Technology (Facul'

# Preprocesamiento de Datos

In [3]:
df = pd.read_excel('../data/raw/QS by Subjects 2014-2021.xlsx',
                   sheet_name='History')


In [4]:
df


,Year,Faculty,Subject,Rank,Institution,Location,Academic Score,Employer Score,Citations Score,H Score,Score,Rank TB,Score_Calc,RankInt,QS Region
0,2014,Arts & Humanities,History,3,Harvard University,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,Arts & Humanities,History,1,University of Cambridge,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,Arts & Humanities,History,2,University of Oxford,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,Arts & Humanities,History,6,"University of California, Berkeley (UCB)",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,Arts & Humanities,History,4,Yale University,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,2020,Arts & Humanities,History,151-200,University of Reading,United Kingdom,55.5,60.5,83.3,82.5,NaN,NaN,NaN,NaN,Europe
1101,2020,Arts & Humanities,History,151-200,University of The Witwatersrand,South Africa,55.6,61.7,85.4,82.5,NaN,NaN,NaN,NaN,Middle East & Africa
1102,2020,Arts & Humanities,History,151-200,University of Warsaw,Poland,68.1,73.0,59.4,56.4,NaN,NaN,NaN,NaN,Europe
1103,2020,Arts & Humanities,History,151-200,Vanderbilt University,United States,57.1,46.3,88.0,81.2,NaN,NaN,NaN,NaN,US & Canada


In [5]:
df.shape


(1105, 15)

In [6]:
df.isna().sum()


Year                  0
Faculty               0
Subject               0
Rank                  1
Institution           0
Location              0
Academic Score       51
Employer Score       51
Citations Score      51
H Score              51
Score               803
Rank TB             304
Score_Calc          304
RankInt            1105
QS Region           904
dtype: int64

Empezamos eliminando los NaNs

In [7]:
df['Academic Score'] = df['Academic Score'].fillna(df['Academic Score'].mean())
df['Employer Score'] = df['Employer Score'].fillna(df['Employer Score'].mean())
df['Citations Score'] = df['Citations Score'].fillna(
    df['Citations Score'].mean())
df['H Score'] = df['H Score'].fillna(df['H Score'].mean())
df['Score'] = df['Score'].fillna(df['Score'].mean())
df = df.drop(columns=['Rank TB', 'Score_Calc', 'RankInt', 'QS Region'])


TypeError: unsupported operand type(s) for +: 'float' and 'datetime.datetime'

In [ ]:
df.isna().sum()


Con todos estos datos haremos un modelo usando los scores para predecir el rank, primeramente necesitamos ver que vairbles estan muy correlacionadas

Tenemos un problema debido a que Rank esta mal formateado, asi que usaremos Score como variable a predecir

In [ ]:
df.corr()['Score'].sort_values()


En este [articulo](https://www.sciencedirect.com/science/article/pii/S0895435615000141) se menciona que solo son necesarias dos variables para la mayoria de modelos de regresion lineal, asi que solo usare Academic y Employer Score

# Creacion del modelo

## Regresion lineal

In [ ]:
cols = ['Academic Score', 'Employer Score', 'Citations Score', 'H Score']


In [ ]:
X = df[cols]
y = df['Score']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)


In [ ]:
modelo_lineal = LinearRegression()
modelo_lineal.fit(X_train, y_train)
predictions = modelo_lineal.predict(X_test)
print('train score', modelo_lineal.score(X_train, y_train))
print('test score', modelo_lineal.score(X_test, y_test))


In [ ]:
print(modelo_lineal.coef_)


## Random Forest

In [ ]:
cols = ['Academic Score', 'Employer Score', 'Citations Score', 'H Score']


In [ ]:
X = df[cols]
y = df['Score']


In [ ]:
modelo_forest = RandomForestRegressor(
    max_depth=10, max_leaf_nodes=750, random_state=0)
modelo_forest.fit(X_train, y_train)
predictions = modelo_forest.predict(X_test)
print('train score', modelo_forest.score(X_train, y_train))
print('test score', modelo_forest.score(X_test, y_test))
#scores = cross_val_score(model, X, Y, cv=10)
#print('Mean score:',scores.mean())


# Resultados TEC

In [ ]:
df_tec = df[df['Institution'] == 'Tecnológico de Monterrey (ITESM)']


In [ ]:
df_tec

In [ ]:
x_tec = df_tec[cols]


In [ ]:
x_tec


In [ ]:
print(modelo_lineal.predict(x_tec))
#print(modelo_forest.predict(x_tec))


In [ ]:
df_2021 = df[df['Year'] == 2021]
score_predecido = modelo_lineal.predict(x_tec)[-1]
print(score_predecido)
df_2021[df_2021['Score'] <= score_predecido +
        .1].sort_values(by='Score', ascending=0).head(10)


Usando el modelo lineal el tec **subiria** a la posicion 120

# Comparacion con la Triada

Veremos si las universidades de la triada suben o bajan usando el modelo lineal

* TEC
* Universidad de los Andes
* UC Chile

## Universidad de los Andes

In [ ]:
df_udelosa = df[df['Institution'] == 'Universidad de Los Andes Colombia']


In [ ]:
df_udelosa


In [ ]:
x_udelosa = df_udelosa[cols]


In [ ]:
score_predecido = modelo_lineal.predict(x_udelosa)[-1]
print(score_predecido)
df_2021[df_2021['Score'] <= score_predecido +
        .1].sort_values(by='Score', ascending=0).head(10)


Aparentemente la U de los Andes __subira__ al nivel 174

## Catolica de chile

In [ ]:
df_catolica = df[df['Institution'] ==
                 'Pontificia Universidad Católica de Chile']


In [ ]:
df_catolica


In [ ]:
x_catolica = df_catolica[cols]


In [ ]:
x_catolica


In [ ]:
score_predecido = modelo_lineal.predict(x_catolica)[-1]
print(score_predecido)
df_2021[df_2021['Score'] <= score_predecido +
        .1].sort_values(by='Score', ascending=0).head(10)


Con base en esto, la Uni catolica __subiria__ a la posicion 106